In [1]:
from glob import glob
import scipy.io
import torch.nn as nn
import torch
import numpy as np
import mne
from features import concatenate_features
from read_data import read_data # <--can read edf 

C:\Users\786me\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%%capture
all_fp=glob('Schizophrenia_Dataset_EDF/*.edf') #<----the path of files containing the dataset 

healthy_fp= [ i for i in all_fp if 'h' in i.split('\\')[1]] #List of path of all healthy subjects
patient_fp= [ i for i in all_fp if 's' in i.split('\\')[1]]#List path of all sick subjects

healthy_epochs_list=[read_data(i) for i in healthy_fp] # (no of healthy subjects) * (No.of epochs/trials, channels, length of signal)
patient_epochs_list=[read_data(i) for i in patient_fp] # # (no of Sick subjects) * (No.of epochs/trials, channels, length of signal)

healthy_epochs_array_lable= [[0]*len(i) for i in healthy_epochs_list ]
patient_epochs_array_lable= [[1]*len(i) for i in patient_epochs_list ]

data_list=healthy_epochs_list+patient_epochs_list
label_list= healthy_epochs_array_lable+ patient_epochs_array_lable
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(data_list),len(label_list),len(groups_list))

In [6]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(groups_list)
data_array=np.moveaxis(data_array,1,2)
print(data_array.shape,label_array.shape,group_array.shape)

(9605, 1000, 19) (9605,) (9605,)


In [7]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold(n_splits=4)
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [8]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
    break
    
lul=train_features.shape
print(lul)

(7048, 1000, 19)


In [ ]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model

def block(input):
  conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
  conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
  conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
  x = concatenate([conv1,conv2,conv3],axis=2)
  return x

input= Input(shape=(lul[1],lul[2]))
block1=block(input)
block2=block(block1)
block3=block(block2)

gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)

In [ ]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_features,train_labels,epochs=100,batch_size=128,validation_data=(val_features,val_labels))

Epoch 1/100
56/56 [==============================] - 30s 436ms/step - loss: 0.6080 - accuracy: 0.6535 - val_loss: 0.6014 - val_accuracy: 0.6777
Epoch 2/100
37/56 [==================>...........] - ETA: 7s - loss: 0.2824 - accuracy: 0.8870

KeyboardInterrupt: 

In [11]:
model.evaluate(val_features,val_labels)

61/61 [==============================] - 3s 43ms/step - loss: 1.7572 - accuracy: 0.6433


[1.7572011947631836, 0.6433350443840027]

In [12]:
val_features.shape

(1943, 1000, 19)

In [13]:
train_features.shape

(7662, 1000, 19)